In [1]:
import os
import logging
import numpy as np
from ase.calculators.vasp import Vasp2
from ase.io import read,write
from ase.eos import EquationOfState

#out_dir = "test"
#if not os.path.isdir(out_dir+"log"):
#    os.system("mkdir -p {}/log".format(out_dir))
#
#logging.basicConfig(format='%(asctime)s : %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p',
#               filename=out_dir+"/log/valet.log", filemode="w",level=logging.INFO)  

In [5]:
class ValetJob:
    def __init__(self,POSCAR_init,out_dir):
        
        if os.path.isfile(POSCAR_init):
            self.poscar_init = POSCAR_init
        else:
            logging.error("Cannot locate POSCAR file as {}".format(POSCAR_init))
            print("ERROR: See log for error.")
            exit()
        self.encut = None
        self.poscar_vol = None
        self.poscar_opt = None
        self.band_structure = None
        self.band_gap = None
        self.out_dir = out_dir
        
        if not os.path.isdir(self.out_dir+"log"):
            os.system("mkdir -p {}/log".format(self.out_dir))

        logging.basicConfig(format='%(asctime)s : %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p',
               filename=out_dir+"/log/valet.log", filemode="w",level=logging.INFO)  
      
    def clean_up(self):
        os.system("cd {}; rm ase* C* D* E* I* K* O* PC* POSCAR POT* vaspr* WA* X*".format(self.out_dir))
    
    def set_encut_autotune_params(self,tol=None,max_steps=None,start_encut=None,encut_step=None):
        stop = False
        ens = []
        cnt = 0
        return tol,stop,cnt,max_steps,start_encut,encut_step,ens
    
    def check_stop(self,x,tol):
        perc_diff = lambda x,y: np.abs(np.abs(x-y)/((x+y)/2))*100
        if len(x) < 2:
            return False
        elif perc_diff(x[-1],x[-2]) < tol:
            return True
        else: 
            return False
        
    def do_encut_autotune(self,tol=1E-4,max_steps=32,start_encut=200,encut_step=25,retune=False):
        logging.info("Commencing with ENCUT autotune.")
        if (self.encut == None) or (retune):
            sys = read(self.poscar_init)
            calc = Vasp2(xc='PBE',kpts=(4,4,4),directory=self.out_dir,atoms=sys)
            tol,stop,cnt,max_steps,ENCUT,ENCUT_step,ens = self.set_encut_autotune_params(tol=tol,max_steps=max_steps,
                                                                                         start_encut=start_encut,
                                                                                         encut_step=encut_step)

            print("Autotuning ENCUT...")
            while not stop:
                if 'error' in locals():
                    del error
                if cnt < max_steps:
                    logging.debug("Count {}".format(cnt))
                    calc.set(encut=ENCUT)
                    ens.append(sys.get_potential_energy())
                    #print(ENCUT)
                    stop = self.check_stop(ens,tol)
                    ENCUT += ENCUT_step
                    cnt += 1
                else:
                    logging.debug("Maximum autotune steps reached.")
                    error = "Maximum autotune steps reached."
                    tip = "Tip: Increase either start_encut, encut_step, or max_steps. Decrease tol."
                    stop = True

            os.system("rm {}/W*".format(self.out_dir))
            try:
                print(error)
                print(tip)
                self.encut = None
            except NameError:
                print("\tENCUT auto-tuned to: {}".format(ENCUT))
                logging.info("Completed ENCUT autotune.")
                self.encut = ENCUT
            print("Done.")  
        else:
            print("ENCUT ready.")
            logging.info("Completed ENCUT autotune.")
        self.clean_up()
        
    def set_volume_autotune_params(self,start_scan=None,end_scan=None,num_scans=None,exclude_type=None,only_type=None):
        vols = []
        ens = []
        return start_scan,end_scan,num_scans,exclude_type,only_type,vols,ens

    def do_volume_autotune(self,start_scan=0.85,end_scan=1.15,num_scans=15,exclude_type=None,only_type=None):
        if not os.path.isfile(os.path.join(self.out_dir,"POSCAR_vol")):
            encut = self.do_encut_autotune()

            logging.info("Commencing volume scan...")
            print("Determining optimal volume...")
            start_scan,end_scan,num_scans,exclude_type,only_type,vols,ens = self.set_volume_autotune_params(
                                                                            start_scan=start_scan,end_scan=end_scan,
                                                                            num_scans=num_scans,exclude_type=exclude_type,
                                                                            only_type=only_type)


            # Load POSCAR
            try:
                sys = read(self.poscar_init)
                calc = Vasp2(xc='PBE',kpts=(4,4,4),directory=self.out_dir,atoms=sys,encut=encut)
                start_cell = sys.get_cell()
            except:
                logging.error("Initial POSCAR file could not be found at {}".format(poscar_init))
                print("Error: See error log for details.")
                return None
            logging.info("Loaded initial POSCAR file.")

            # Do volume scan
            logging.info("Performing volume scan.")
            for x in np.linspace(start_scan,end_scan,num_scans):
                sys.set_cell(x*start_cell,scale_atoms=True)
                sys.set_calculator(calc)
                ens.append(sys.get_potential_energy())
                vols.append(sys.get_volume())
            logging.info("Volume scan complete.")

            # Fit EoS
            logging.info("Fitting equations of state.")
            eos_types = "sjeos taylor murnaghan birch birchmurnaghan pouriertarantola p3 antonschmidt".split()

            if exclude_type != None:
                _ = [eos_types.remove(i) for i in exclude]
            elif only_type != None:
                eos_types = only_type

            eos_fits = {}
            for typ in eos_types:
                eos = EquationOfState(vols,ens,eos=typ)
                v, e, B = eos.fit()
                eos_fits[typ] = {'volume':v,'energy':e,'buld_modulus':B}

            # Rescale initial cell to optimized volume
            logging.info("Rescaling with optimized volume.")
            vol_avg = np.average([eos_fits[key]['volume'] for key in eos_fits.keys()])
            sys.set_cell(start_cell,scale_atoms=True)

            scale = sys.get_volume()/vol_avg
            sys.set_cell(scale*start_cell,scale_atoms=True)

            # Perform sc-step
            logging.info("Performing self-consistent step.")
            sys.set_calculator(calc)
            en = sys.get_potential_energy()

            # Save structure
            logging.info("Saving final POSCAR_vol.")
            self.poscar_vol = os.path.join(self.out_dir,"POSCAR_vol")
            write(self.poscar_vol,sys)

            print("Done.")
            logging.info("Volume scan complete.")
            self.clean_up()
        else:
            sys = read(os.path.join(self.out_dir,"POSCAR_vol"))
            print("Volume optimized system ready.")
        

In [3]:
a = ValetJob("POSCAR.mp-27_Si","auto_test")

In [4]:
a.do_volume_autotune()

Autotuning ENCUT...
	ENCUT auto-tuned to: 725
Done.
Determining optimal volume...
Done.
